In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [4]:
%load_ext autoreload
%autoreload 2

% cd /gdrive/My\ Drive/Colab\ Notebooks/nlp_task/kenlm
! pwd
! ls

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/gdrive/My Drive/Colab Notebooks/nlp_task/kenlm
/gdrive/My Drive/Colab Notebooks/nlp_task/kenlm
corpus_task  insurance_train.txt  train_lm.ipynb  train.txt.gz


In [3]:
# import gzip, os, shutil

# with gzip.GzipFile('train.txt.gz') as zipf:
#     open('insurance_train.txt', "wb+").write(zipf.read())     

In [5]:
# import requests

# request_2 = requests.get("https://github.com/Computing-Intelligence/datasource/raw/master/movie_comments.csv")
# with open('movie_comments.csv', 'wb') as f:
#     f.write(request_2.content)

In [6]:
# 安装依赖
!apt install libboost-all-dev
!apt install libbz2-dev
!apt install libeigen3-dev

# 下载KenLM编译
!wget -O - https://kheafield.com/code/kenlm.tar.gz | tar xz
!mkdir kenlm/build
!cd kenlm/build && cmake .. && make -j8

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libboost-all-dev is already the newest version (1.65.1.0ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 59 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libbz2-dev is already the newest version (1.0.6-8.1ubuntu0.2).
libbz2-dev set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 59 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
Suggested pa

In [7]:
!head -5 ./corpus_task

 我 们 去 抢 劫 吧
 我 们 去 抢 劫 吧 哎 ， 我 最 腻 味 这 种 没 有 技 术 含 量 的 活 儿 ～
 王 。
 王 。 哎 ， 叫 朕 何 事 ？
 喜 欢 什 么 电 影


In [10]:
!pwd

/gdrive/My Drive/Colab Notebooks/nlp_task/kenlm


In [11]:
# 3-gram语言模型
!./kenlm/build/bin/lmplz -o 3 <corpus_task> lm_3.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /gdrive/My Drive/Colab Notebooks/nlp_task/kenlm/corpus_task
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
tcmalloc: large alloc 2514853888 bytes == 0x55a4e4348000 @  0x7fad943651e7 0x55a4e18c3772 0x55a4e1857358 0x55a4e1836290 0x55a4e1822096 0x7fad924feb97 0x55a4e1823ada
tcmalloc: large alloc 8382840832 bytes == 0x55a57a1a2000 @  0x7fad943651e7 0x55a4e18c3772 0x55a4e18ad7aa 0x55a4e18ae1c8 0x55a4e18362ad 0x55a4e1822096 0x7fad924feb97 0x55a4e1823ada
****************************************************************************************************
Unigram tokens 3073460 types 4871
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:58452 2:3799231744 3:7123559424
tcmalloc: large alloc 7123566592 bytes == 0x55a4e4348000 @  0x7fad943651e7 0x55a4e18c3772 0x55a4e18ad7aa 0x55a4e18ae1c8 0x55a4e183684e 0x55a4e1822096 0x7fad924feb97 0x55a4e1823ada
tcmalloc: large alloc 379923660

对训练得到的文件进行压缩：将arpa文件转换为binary文件，这样可以对arpa文件进行压缩和序列化，提高后续在python中加载的速度。针对我们训练的到的 lm.arpa 文件其转换命令为：

In [12]:
!./kenlm/build/bin/build_binary -s lm_3.arpa lm.bin

Reading lm_3.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


In [13]:
!pip install https://github.com/kpu/kenlm/archive/master.zip

     \ 2.8MB 391kB/s
  Created wheel for kenlm: filename=kenlm-0.0.0-cp36-cp36m-linux_x86_64.whl size=2330547 sha256=e2d49e0206abd28c20bc8d413d5445226259003d33383b518fd48c00578666b1
  Stored in directory: /tmp/pip-ephem-wheel-cache-m9lk4anj/wheels/2d/32/73/e3093c9d11dc8abf79c156a4db1a1c5631428059d4f9ff2cba
Successfully built kenlm


In [14]:
# 加载模型
import kenlm

# 将文件导入到 kenlm 语言模型中
model = kenlm.LanguageModel("./lm.bin")

In [15]:
model.score("怎 么 办 理 信 用 卡")

-7.646023750305176

In [16]:
model.score("怎 么 办 理 新 用 卡")

-14.39771556854248

In [17]:
model.score("怎 么 办 理 储 蓄 卡")

-8.711942672729492

In [18]:
model.score("龙 卡 是 什 么")

-8.195541381835938

In [19]:
model.score("农 卡 是 什 么")

-12.113533020019531

In [20]:
model.score("卡 农 是 什 么")

-12.968329429626465

Nice

## 拼音转换

In [8]:
from pypinyin import pinyin, lazy_pinyin, Style

In [7]:
pinyin('中心')

[['zhōng'], ['xīn']]

In [3]:
pinyin('中心', heteronym=True)  # 启用多音字模式

[['zhōng', 'zhòng'], ['xīn']]

In [4]:
pinyin('中心', style=Style.FIRST_LETTER)  # 设置拼音风格

[['z'], ['x']]

In [5]:
pinyin('中心', style=Style.TONE2, heteronym=True)

[['zho1ng', 'zho4ng'], ['xi1n']]

In [8]:
pinyin('中心', style=Style.TONE3, heteronym=True)

[['zhong1', 'zhong4'], ['xin1']]

In [9]:
pinyin('中心', style=Style.BOPOMOFO)  # 注音风格

[['ㄓㄨㄥ'], ['ㄒㄧㄣ']]

In [11]:
lazy_pinyin('银行')  # 不考虑多音字的情况

['yin', 'hang']

### Trie测试

In [9]:
# import os
# import stat
# import sys

# import pygtrie

https://github.com/ethereum/py-trie bug一堆

In [10]:
import string
import datrie

In [12]:
trie = datrie.Trie(string.ascii_lowercase)

In [13]:
trie[u'foo'] = 5

In [14]:
trie[u'foo'] = 6

In [15]:
trie[u'foo']

6

In [16]:
s = u'你好'.encode('utf-8')

In [17]:
print(s)

b'\xe4\xbd\xa0\xe5\xa5\xbd'


In [18]:
import json

json.dumps('你好')

'"\\u4f60\\u597d"'

In [19]:
'你好'.encode("unicode_escape").decode()

'\\u4f60\\u597d'

## 处理语料

In [1]:
from utils import save_pickle

In [20]:
import re

def token(string):
        return re.findall(r'[\d|\w]+', string)

In [96]:
words = {}
words_frequnece = {}
# 文本文件经过预处理: language_model_data_process
with open('data/corpus_task', 'r', encoding='utf-8') as f:
    for line in f:
        line = token(line)
#         print(line)
#         break
        for w in line:
            if w in words: 
                words_frequnece[w] = words_frequnece[w] + 1
                continue
            words_frequnece[w] = 1
            pin = lazy_pinyin(w)[0]
            words[w] = pin

In [99]:
save_pickle('data/words_frequnece.bin', words_frequnece)

In [95]:
# def ch_char(string):
#     return re.findall(r'[\u4e00-\u9fa5]', string)

# 增加一点汉字集, 非必要
# with open('data/raw_article_9k.txt', 'r', encoding='utf-8') as f:
# #     print(f.readline())
# #     print(ch_char(f.readline()))
#     for line in f.readlines():
#         line = ch_char(line)
#         for w in line:
#             if w in words: continue
#             pin = lazy_pinyin(w)[0]
#             words[w] = pin

In [22]:
import string
import datrie

In [23]:
trie = datrie.Trie(string.ascii_lowercase + string.digits)

In [24]:
for w, pin in words.items():
    if pin in trie and w == trie[pin]:
        continue
    # 相同拼音处理
    if pin in trie and w != trie[pin]:
        i = 1
        flag = True
        while flag:
            key = pin + str(i)
            if key not in trie:
                flag = False
            i += 1
        trie[key] = w
    else:
        trie[pin] = w

In [25]:
trie.prefixes('xi')

['x', 'xi']

In [26]:
trie.suffixes('yong')

['', '1', '10', '2', '3', '4', '5', '6', '7', '8', '9']

In [52]:
trie.suffixes('wen')

['', '1', '10', '2', '3', '4', '5', '6', '7', '8', '9', 'g']

In [480]:
string.ascii_lowercase

'abcdefghijklmnopqrstuvwxyz'

In [481]:
a = [1,2,3,1]

In [61]:
trie['wen']

'紊'

In [62]:
a = [1,2,3]
a[:10]

[1, 2, 3]

## 文本纠错

#### NOTE

一个问题在于，如果文本纠错是一个字一个字替换，然后根据LM打分，那么输入文本较长时，搜索空间指数级增长

由于在retrival阶段，会按照重要名词进行布尔搜索，所以此处只对名词性词汇进行纠错

In [31]:
# 加载模型
import kenlm

# 将文件导入到 kenlm 语言模型中
model = kenlm.LanguageModel("./model/lm.bin")

In [37]:
from itertools import product

#### 增加词频限制

In [101]:
from functools import reduce


avg = reduce(lambda a, b: a + b, words_frequnece.values()) / len(words_frequnece)

In [102]:
avg

576.2143456375838

In [111]:
def find_word(string):
    pattern = re.compile(r'[\u4e00-\u9fa5]|[abcdefghijklmnopqrstuvwxyz]+')
    return re.findall(pattern, string.lower())
    
    
def space_split(string):
    pre = '@'
    line = ''
    for s in string:
        if ord('A') <= ord(pre) <= ord('z') and  ord('A') <= ord(s) <= ord('z'):
            line += s
        else:
            line += ' ' + s
        pre = s
    return line
    
    
def check_english(w):
    return (ord('a') <= ord(w[0]) <= ord('z')) and ((ord('a') <= ord(w[-1]) <= ord('z')))
    

def find_candidates(w):
    candidates = set()
    pin = lazy_pinyin(w)[0]
    try:
        # prefix查找
        not_leaf = list(filter(lambda x: 0 <= (len(pin) - len(x)) <= 1, trie.prefixes(pin)))

        if len(not_leaf) == 0:
            not_leaf = trie.prefixes(pin)[0]
        
        for prefix in not_leaf:
            suffixes = trie.suffixes(prefix)
            for suf in suffixes:
                cand = prefix + suf
                # 长度限制
                if 0 <= len(cand) - len(pin) <= 1:
                    candidates.add(trie[cand])
                    
        candidates.add(w)
    except KeyError:
        pass
    return candidates


def frequence_filter(possible_set):
    return sorted(list(possible_set), key=lambda w: words_frequnece.get(w, 0), reverse=True)[:5]

    
def creat_query(self, query, to_create):
    start_idx = 0
    words_corrected = []
    count = 0
    for word in to_create:
        idx = query.index(word, start_idx)
        best_lm_score = -10000
        
        possible_w = []
        if check_english(word):  # 英文直接找
            possibles = frequence_filter(find_candidates(word))
        else:
            for w in word:  # 汉语一个字一个字替换
                filter_words = frequence_filter(find_candidates(w))
                possible_w.append(filter_words)
                
            possibles = product(*possible_w)
        
        candi = ''
        
        for repl in possibles:
#             print(repl)
            count += 1
            tmp = query[: start_idx] +  query[start_idx: ].replace(word, ''.join(repl), 1)
            score = model.score(space_split(tmp))
            if score > best_lm_score: 
                best_lm_score = score
                candi = tmp
        query = candi
        words_corrected.append(query[idx: idx + len(word)])
        start_idx = idx + len(word)
    print(count)
    return query, words_corrected
    
    
#     words = find_word(query)
#     return words
    

In [33]:
model.score("请 问 怎 么 办 理 新 永 卡")

-20.38299560546875

In [34]:
model.score("请 问 怎 么 办 理 瑕 镛 卡")

-23.79347038269043

In [35]:
model.score("请 问 怎 么 办 理 信 用 卡")

-11.427366256713867

In [117]:
%%timeit
creat_query(1, "请吻办vasa，需要深粉证吗, 请问怎么办了新永卡，你好哈,请吻办vasa，需要深粉证吗, 请问怎么办了新永卡，你好哈,请吻办vasa，需要深粉证吗, 请问怎么办了新永卡，你好哈,请吻办vasa，需要深粉证吗, 请问怎么办了新永卡，你好哈", 
            ["vasa", "深粉证", "新永卡","vasa", "深粉证", "新永卡", "vasa", "深粉证", "新永卡", "vasa", "深粉证", "新永卡"])

1016


('请吻办visa，需要身份证吗, 请问怎么办了信用卡，你好哈,请吻办visa，需要身份证吗, 请问怎么办了信用卡，你好哈,请吻办visa，需要身份证吗, 请问怎么办了信用卡，你好哈,请吻办visa，需要身份证吗, 请问怎么办了信用卡，你好哈',
 ['visa',
  '身份证',
  '信用卡',
  'visa',
  '身份证',
  '信用卡',
  'visa',
  '身份证',
  '信用卡',
  'visa',
  '身份证',
  '信用卡'])

In [121]:
creat_query(1, "请吻办vasa，需要深粉证吗, 请问怎么办了新永卡，你好哈", ["请","吻", "vasa", "深粉证", "新","永","卡", "你", "好"])

164


('请问办visa，需要身份证吗, 请问怎么办了下用卡，你好哈',
 ['请', '问', 'visa', '身份证', '下', '用', '卡', '你', '好'])

In [1]:
from nlp import Ltp

In [2]:
ltp = Ltp(pos=True, seg_lexicon_path='data/lexicon_seg.txt', pos_lexicon_path='data/lexicon.txt')

In [118]:
a = ltp.segmentor.segment("请吻怎么伴离新永卡")
print(list(a))
b = ltp.postagger.postag(a)
print(list(b))

['请', '吻', '怎么', '伴', '离', '新', '永卡']
['v', 'v', 'r', 'v', 'v', 'a', 'n']


In [352]:
to_create = ["新永卡", "银行", "vise"]

In [491]:
# test
# possible_w = []
# for word in to_create:
#     if check_english(word):
#         possible_w.append(find_candidates(word))
#     else:
#         for w in word:
#             possible_w.append(find_candidates(w))

# print(possible_w)

In [2]:
from utils import *

In [8]:
word_freq = unpickle_file('data/full_word2id.bin')

In [9]:
vocab = set()
for k, v in word_freq.items():
    vocab.add(k)

In [10]:
len(vocab)

11320

In [11]:
save_pickle('data/vocabulary.bin', vocab)

## 保存

In [488]:
from utils import save_pickle

In [489]:
save_pickle('model/create_datrie.bin', trie)